In [1]:
import pandas as pd
import sqlite3
import requests
import numpy as np
import folium
import folium.plugins 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
con = sqlite3.connect('Campsite_data_4k-all.db')
query = ''' SELECT * FROM Venues'''
completed_venues = pd.read_sql_query(query,con).drop('index',axis=1)
query = ''' SELECT Campsite_Name, Latitude, Longitude FROM Sites WHERE Campsite_Name not like '%closed%'   '''
df_RV = pd.read_sql_query(query,con).drop_duplicates()
con.close()
Camp_sites_onehot = pd.get_dummies(completed_venues[['Venue_Category']], prefix="", prefix_sep="")
Camp_sites_onehot['Latitude'] = completed_venues['Latitude']
Camp_sites_onehot = Camp_sites_onehot[[Camp_sites_onehot.columns[-1]] + list(Camp_sites_onehot.columns[:-1])]
campsite_grouped = Camp_sites_onehot.groupby('Latitude').mean().reset_index()
campsite_grouped_clustering = campsite_grouped.drop('Latitude', 1)
campsite_grouped

,Latitude,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Well,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,19.136683,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.501292,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19.734633,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.026883,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21.071786,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9040,64.962006,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9041,64.997376,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9042,65.053898,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9043,65.825600,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_RV

,Campsite_Name,Latitude,Longitude
0,'KSAN Historical Village Campground,55.249428,-127.679933
1,10 Acres Campground and RV Park,44.030277,-73.403468
2,1000 Islands Campground,44.276788,-75.997158
3,1000 Islands Camping Resort,44.350050,-76.054170
4,1000 Islands/Association Island K O A,43.891877,-76.216943
...,...,...,...
16972,Zippel Bay SP,48.847923,-94.849806
16973,Zooland Family Campground,35.603549,-79.850677
16974,Zumbro Bottoms Horse Campground,44.301126,-92.122085
16975,Zuni Village RV Park,35.224768,-114.014515


In [4]:
kmeans = KMeans(n_clusters=12, random_state=0).fit(campsite_grouped_clustering)

In [5]:
kmeans.labels_[0:30]

array([6, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 6, 0, 6, 0,
       0, 0, 0, 0, 0, 0, 4, 7], dtype=int32)

In [6]:
campsite_grouped.insert(0, 'Kmeans_Cluster_Labels', kmeans.labels_)
df_RV = df_RV.reset_index().set_index('Latitude')
campsite_grouped = campsite_grouped.set_index('Latitude')

In [7]:
df_RV['Kmeans_Cluster_Labels'] = campsite_grouped['Kmeans_Cluster_Labels']
df_RV = df_RV.reset_index()
df_RV

,Latitude,index,Campsite_Name,Longitude,Kmeans_Cluster_Labels
0,55.249428,0,'KSAN Historical Village Campground,-127.679933,0.0
1,44.030277,1,10 Acres Campground and RV Park,-73.403468,0.0
2,44.276788,2,1000 Islands Campground,-75.997158,0.0
3,44.350050,3,1000 Islands Camping Resort,-76.054170,7.0
4,43.891877,4,1000 Islands/Association Island K O A,-76.216943,0.0
...,...,...,...,...,...
16971,48.847923,16972,Zippel Bay SP,-94.849806,NaN
16972,35.603549,16973,Zooland Family Campground,-79.850677,NaN
16973,44.301126,16974,Zumbro Bottoms Horse Campground,-92.122085,NaN
16974,35.224768,16975,Zuni Village RV Park,-114.014515,NaN


In [8]:
df_RV.groupby('Kmeans_Cluster_Labels').count()

,Latitude,index,Campsite_Name,Longitude
Kmeans_Cluster_Labels,,,,
0.0,3116,3116,3116,3116
1.0,532,532,532,532
2.0,89,89,89,89
3.0,674,674,674,674
4.0,2629,2629,2629,2629
5.0,99,99,99,99
6.0,158,158,158,158
7.0,1041,1041,1041,1041
8.0,280,280,280,280


In [9]:
# create map of RV parks using latitude and longitude values
latitude = df_RV['Latitude'].mean()
longitude = df_RV['Longitude'].mean()
map_national_RV_parks_clusters = {}
for cluster in df_RV.groupby('Kmeans_Cluster_Labels').count().reset_index()['Kmeans_Cluster_Labels'].tolist():
    
    map_national_RV_parks_clusters[cluster] = folium.Map(location=[latitude, longitude], zoom_start=4)

    # add markers to map
    marker_cluster_clusters = {}
    marker_cluster_clusters[cluster] = folium.plugins.MarkerCluster().add_to(map_national_RV_parks_clusters[cluster])
    for lat, lng, name in zip(df_RV[df_RV['Kmeans_Cluster_Labels']==cluster]['Latitude'], df_RV[df_RV['Kmeans_Cluster_Labels']==cluster]['Longitude'], df_RV.reset_index()[df_RV['Kmeans_Cluster_Labels']==cluster]['Campsite_Name']):
        label = "<a  target='_blank' href='https://www.google.com/search?q={}'>{}</a>".format(name.replace(' ','%20'),name)
        #label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(marker_cluster_clusters[cluster])  

    map_national_RV_parks_clusters[cluster].save('Kmeans/Cluster {} from analysis.html'.format(cluster))

In [10]:
df_RV = df_RV.drop('index',axis=1)

In [11]:
con = sqlite3.connect('Campsite_data_4k-all.db')
df_RV.to_sql('Kmeans',con,if_exists='replace')
campsite_grouped.reset_index().to_sql('Kmeans_grouped',con,if_exists='replace')
con.close()

/home/theia/.local/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Let's make sure we can read the data

In [12]:
con = sqlite3.connect('Campsite_data_4k-all.db')
query = ''' SELECT * FROM Kmeans'''
df_RV_Kmeans = pd.read_sql_query(query,con).drop('index',axis=1)
query = ''' SELECT * FROM Kmeans_grouped'''
df_RV_Kmeans_grouped = pd.read_sql_query(query,con).drop('index',axis=1)
con.close()

In [13]:
df_RV_Kmeans

,Latitude,Campsite_Name,Longitude,Kmeans_Cluster_Labels
0,55.249428,'KSAN Historical Village Campground,-127.679933,0.0
1,44.030277,10 Acres Campground and RV Park,-73.403468,0.0
2,44.276788,1000 Islands Campground,-75.997158,0.0
3,44.350050,1000 Islands Camping Resort,-76.054170,7.0
4,43.891877,1000 Islands/Association Island K O A,-76.216943,0.0
...,...,...,...,...
16971,48.847923,Zippel Bay SP,-94.849806,NaN
16972,35.603549,Zooland Family Campground,-79.850677,NaN
16973,44.301126,Zumbro Bottoms Horse Campground,-92.122085,NaN
16974,35.224768,Zuni Village RV Park,-114.014515,NaN


In [14]:
list(range(0,int(df_RV_Kmeans['Kmeans_Cluster_Labels'].max())+1,1))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## I will now seperate the dataFrame into seperate frames

In [15]:
df_Kmeans = {}
df_Kmeans_grouped = {}
df_Kmeans_stats = {}
for cluster in list(range(0,int(df_RV_Kmeans['Kmeans_Cluster_Labels'].max())+1,1)):
    df_Kmeans[cluster] = df_RV_Kmeans[df_RV_Kmeans['Kmeans_Cluster_Labels'] == cluster]
    df_Kmeans_grouped[cluster] = df_RV_Kmeans_grouped[df_RV_Kmeans_grouped['Kmeans_Cluster_Labels'] == cluster]
    cols = (df_Kmeans_grouped[cluster] != 0).any()
    try:
        df_Kmeans_grouped[cluster] = df_Kmeans_grouped[cluster][cols[cols].index]
    except:
        pass
    try:
        df_Kmeans_stats[cluster] = df_Kmeans_grouped[cluster].drop(['Kmeans_Cluster_Labels','Latitude'],1).describe().T.sort_values(by='mean', ascending=False)
    except:
        df_Kmeans_stats[cluster] = df_Kmeans_grouped[cluster].drop(['Latitude'],1).describe().T.sort_values(by='mean', ascending=False)
        #df_Kmeans_stats[cluster] = df_Kmeans_grouped[cluster].drop(['Kmeans_Cluster_Labels','Latitude'],1).describe().T.sort_values(by='mean', ascending=False)
    print("Cluster number: {}, cluster size: {}, number of venue types: {}".format(cluster, df_Kmeans_grouped[cluster].shape[0], df_Kmeans_grouped[cluster].shape[1]-1))

Cluster number: 0, cluster size: 3110, number of venue types: 540
Cluster number: 1, cluster size: 531, number of venue types: 145
Cluster number: 2, cluster size: 89, number of venue types: 74
Cluster number: 3, cluster size: 673, number of venue types: 246
Cluster number: 4, cluster size: 2620, number of venue types: 531
Cluster number: 5, cluster size: 99, number of venue types: 40
Cluster number: 6, cluster size: 158, number of venue types: 131
Cluster number: 7, cluster size: 1040, number of venue types: 326
Cluster number: 8, cluster size: 279, number of venue types: 152
Cluster number: 9, cluster size: 57, number of venue types: 51
Cluster number: 10, cluster size: 320, number of venue types: 179
Cluster number: 11, cluster size: 69, number of venue types: 42


In [16]:
df_Kmeans[0]

,Latitude,Campsite_Name,Longitude,Kmeans_Cluster_Labels
0,55.249428,'KSAN Historical Village Campground,-127.679933,0.0
1,44.030277,10 Acres Campground and RV Park,-73.403468,0.0
2,44.276788,1000 Islands Campground,-75.997158,0.0
4,43.891877,1000 Islands/Association Island K O A,-76.216943,0.0
10,62.387044,1202 Motor Inn and RV Park,-140.873463,0.0
...,...,...,...,...
15491,46.893879,Trapper's Kettle Restaurant and Campground,-103.189784,0.0
15494,48.203390,Travel America RV Park,-116.566229,0.0
15502,42.660486,Traveler's Woods of New England,-72.545573,0.0
15504,29.374874,Traveler's World RV Resort,-98.481504,0.0


In [17]:
for i in df_Kmeans_stats:
    print("Group: {}, DF: {}".format(i,df_Kmeans_stats[i]))

Group: 0, DF:                             count      mean       std  min  25%  50%  \
American Restaurant        3110.0  0.047241  0.082283  0.0  0.0  0.0   
Hotel                      3110.0  0.038915  0.087915  0.0  0.0  0.0   
Restaurant                 3110.0  0.024335  0.078988  0.0  0.0  0.0   
Convenience Store          3110.0  0.024326  0.061831  0.0  0.0  0.0   
Bar                        3110.0  0.021227  0.066257  0.0  0.0  0.0   
...                           ...       ...       ...  ...  ...  ...   
Hospital                   3110.0  0.000005  0.000299  0.0  0.0  0.0   
Indian Chinese Restaurant  3110.0  0.000005  0.000299  0.0  0.0  0.0   
Trade School               3110.0  0.000005  0.000299  0.0  0.0  0.0   
Boxing Gym                 3110.0  0.000005  0.000299  0.0  0.0  0.0   
Jazz Club                  3110.0  0.000005  0.000299  0.0  0.0  0.0   

                                75%       max  
American Restaurant        0.071429  1.000000  
Hotel                    

In [18]:
df_Kmeans_stats[0]

,count,mean,std,min,25%,50%,75%,max
American Restaurant,3110.0,0.047241,0.082283,0.0,0.0,0.0,0.071429,1.000000
Hotel,3110.0,0.038915,0.087915,0.0,0.0,0.0,0.050000,1.000000
Restaurant,3110.0,0.024335,0.078988,0.0,0.0,0.0,0.000000,1.000000
Convenience Store,3110.0,0.024326,0.061831,0.0,0.0,0.0,0.000000,0.500000
Bar,3110.0,0.021227,0.066257,0.0,0.0,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...
Hospital,3110.0,0.000005,0.000299,0.0,0.0,0.0,0.000000,0.016667
Indian Chinese Restaurant,3110.0,0.000005,0.000299,0.0,0.0,0.0,0.000000,0.016667
Trade School,3110.0,0.000005,0.000299,0.0,0.0,0.0,0.000000,0.016667
Boxing Gym,3110.0,0.000005,0.000299,0.0,0.0,0.0,0.000000,0.016667


In [19]:
range_i = list(range(1,campsite_grouped['Kmeans_Cluster_Labels'].max()+2))
df_clusters = []
df_clusters_stats = []
top_ten = []
for cluster in range_i:
    df_clusters.append(campsite_grouped[campsite_grouped['Kmeans_Cluster_Labels'] == cluster-1])
    cols = (df_clusters[-1] != 0).any()
    df_clusters[-1] = df_clusters[-1][cols[cols].index]
    try:
        df_clusters_stats.append(df_clusters[-1].drop(['Kmeans_Cluster_Labels'],1).describe().T.sort_values(by='mean', ascending=False))
    except:
        df_clusters_stats.append(df_clusters[-1].describe().T.sort_values(by='mean', ascending=False))
    temp = {}
    temp['name'] = 'Cluster {}'.format(cluster-1)
    lo = 0
    for top in df_clusters_stats[-1].iloc[:10].reset_index()['index'].tolist():
        lo = lo+1
        temp[str(lo)] = top
    top_ten.append(temp)
    print(cluster)
df_top_ten = pd.DataFrame(top_ten).set_index(['name'])
df_top_ten

1
2
3
4
5
6
7
8
9
10
11
12


,1,2,3,4,5,6,7,8,9,10
name,,,,,,,,,,
Cluster 0,American Restaurant,Hotel,Restaurant,Convenience Store,Bar,Harbor / Marina,Scenic Lookout,Park,Seafood Restaurant,Resort
Cluster 1,Campground,Lake,State / Provincial Park,Home Service,Scenic Lookout,Beach,Construction & Landscaping,Trail,American Restaurant,Convenience Store
Cluster 2,Park,Campground,State / Provincial Park,Lake,Convenience Store,Home Service,Hotel,Golf Course,River,Trail
Cluster 3,Lake,Campground,Harbor / Marina,State / Provincial Park,American Restaurant,Trail,Construction & Landscaping,Scenic Lookout,Convenience Store,Golf Course
Cluster 4,Fast Food Restaurant,Gas Station,Sandwich Place,Pizza Place,Discount Store,Convenience Store,Grocery Store,Coffee Shop,American Restaurant,Mexican Restaurant
Cluster 5,Lake,Campground,Harbor / Marina,State / Provincial Park,Burger Joint,Beach,Resort,Tourist Information Center,Rest Area,Park
Cluster 6,Beach,Campground,Harbor / Marina,State / Provincial Park,Resort,Scenic Lookout,Trail,National Park,Seafood Restaurant,Park
Cluster 7,Campground,Trail,American Restaurant,Park,Scenic Lookout,Convenience Store,Construction & Landscaping,Home Service,State / Provincial Park,Farm
Cluster 8,Trail,Campground,Scenic Lookout,State / Provincial Park,Mountain,National Park,Park,Tourist Information Center,Lake,American Restaurant


In [20]:
con = sqlite3.connect('Campsite_data_4k-all.db')
df_top_ten.reset_index().to_sql('Kmeans_top_ten',con,if_exists='replace')
con.close()